## 2019年7月9日
我们要做什么呢？
* 首先，为每一个module找到它们的reference

In [1]:
import json, md2py, re
from lxml import etree

JSON_FILE = r"J:\ModelStoreData\AIHub\2019-06-04\TFModule_Info.json"

def beautify_string(paper):
    paper = paper.replace("\"", "")
    paper = paper.replace("\n", " ").strip()
    paper = ' '.join(paper.split())
    return paper

# def remove_github_items(l):
#     for ll in l: 
#         if "GitHub issue" in ll:
#             l.remove(ll)
#         if "https://github.com" in ll:
#             l.remove(ll)
#     return l

def remove_github_items(keywords, l):
    removed_index = []
    for i in range(len(l)): 
        if keywords in l[i]:
            removed_index.append(i)
    for i in sorted(removed_index, reverse=True):
        del l[i]
    return l
#         print(re.findall("[G, g]it[H, h]ub"))

paper_link = {}
module_link = {}
paper_module = {}
module_paper = {}
module_version = {}

def get_reference_info(module, md):
    toc = md2py.md2py(md)
    page_source = etree.HTML(str(toc.source))
    papers = page_source.xpath("//li/a/text()")
    links = page_source.xpath("//li/a/@href")
    
    ## the amoebanet modules may have strange paper name which is "arXiv:1802.01548". But the paper name can be got manually. 
    ## lets' omit this detail first. 
    ## if necessary, make some change. 
#     if "amoebanet" in module:
#         pass        
    
    papers = list(map(beautify_string, papers))
    papers = remove_github_items("GitHub issue", papers)
    links = remove_github_items("https://github.com", links)

    for paper, link in zip(papers, links):
        paper_link[paper] = link

        if paper not in paper_module:
            paper_module[paper] = [module]
        else:
            if module not in paper_module[paper]:
                paper_module[paper].append(module)
#             if module not in module_paper:
#                 module_paper[module] = [paper]
#             else:
#                 if paper not in module_paper[module]:
#                     module_paper[module].append(paper)    
    module_paper[module] = papers
    
def add_paper_into_dicts(paper, module):
    if paper not in paper_module:
        paper_module[paper] = [module]
    else:
        if module not in paper_module[paper]:
            paper_module[paper].append(module)

def getting_information():
    with open(JSON_FILE, "r") as jf:
        items = json.load(jf)
    for item in items:
        module = item["name"]
        version = item["version"]
        info = item["info"]
        link_url = "https://aihub.cloud.google.com/p/products%2F{}".format(info[1][1].split("/")[1])
        md = info[1][7]

        if module in module_version:
            module_version[module].append(version)
        else:
            module_version[module] = [version]

        module_link[module] = link_url

        if "imagenet" in module.split("-")[0]:    
            get_reference_info(module, md)
        elif "eference" in md:
            toc = md2py.md2py(md)
            page_source = etree.HTML(str(toc.source))
            ## paper name could be in the following two xpathes.
    #         combi1 = page_source.xpath("//p[last()]/text()")
    #         print(combi1)
            combi2 = page_source.xpath("//a[last()]/text()")
            ## the best part is in "paper"
            if "universal-sentence-encoder-xling" in module:
                paper1 = "Learning Cross-Lingual Sentence Representations via a Multi-task Dual-Encoder Model." ## combi1 will be: ['[1] M. Chidambaram, Y. Yang, D. Cer, S. Yuan, Y.-H. Sung, B. Strope, and R.\nKurzweil. Learning Cross-Lingual Sentence Representations via a Multi-task\nDual-Encoder Model. ArXiv e-prints, October 2018.']
                add_paper_into_dicts(paper1, module)
                paper2 = "Universal Sentence Encoder"
                add_paper_into_dicts(paper2, module)
                module_paper[module] = [paper1, paper2]
            elif "delf" in module:
                papers = list(map(beautify_string, combi2))
                for paper in papers:
                    add_paper_into_dicts(paper, module)
                module_paper[module] = papers
            else:
                paper = combi2[-1].replace("\n", "")
                add_paper_into_dicts(paper, module)
                module_paper[module] = [paper]
        else:
            if "tf2-preview" in module:
                get_reference_info(module, md)
            elif "inaturalist-inception_v3" in module:
                get_reference_info(module, md)
            else:
                print(module, version, link_url)
                pass

getting_information()

ganeval-cifar10-convnet 1 https://aihub.cloud.google.com/p/products%2F110689d8-c594-49d6-aef4-9fc91c9c2ab4
faster_rcnn-openimages_v4-inception_resnet_v2 1 https://aihub.cloud.google.com/p/products%2F41b42dfa-e600-4a73-a425-7c5c4d511c3c
random-nnlm-en-dim50 1 https://aihub.cloud.google.com/p/products%2Fb0766fd5-82b5-48e3-9d6f-26c5d2601d5f
image_augmentation-crop_rotate_color 1 https://aihub.cloud.google.com/p/products%2Fcd9eda73-8cc4-4dbc-aa62-c97e98d657af
image_augmentation-nas_svhn 1 https://aihub.cloud.google.com/p/products%2F5eef4456-8aa9-47d7-84db-00a7faa6662a
image_augmentation-crop_color 1 https://aihub.cloud.google.com/p/products%2F83bb37d7-5060-4ba7-85e4-6c00a582e789
random-nnlm-en-dim128 1 https://aihub.cloud.google.com/p/products%2Fa52e9466-f691-4f91-a62e-f6e39eb967f8
image_augmentation-nas_cifar 1 https://aihub.cloud.google.com/p/products%2F822fa7f1-2207-4645-bcab-b7b916dae368
image_augmentation-nas_imagenet 1 https://aihub.cloud.google.com/p/products%2Fdaf153b0-cc1b-4f27-a0

In [2]:
def add_paper_and_module(papers, modules):
    for paper in papers:
        paper_module[paper] = modules
    for module in modules:
        module_paper[module] = papers
add_paper_and_module(
    ["AutoAugment: Learning Augmentation Policies from Data"], 
    [
        "image_augmentation-nas_svhn", 
        "image_augmentation-nas_imagenet", 
        "image_augmentation-nas_cifar"
    ]
)
paper_link["AutoAugment: Learning Augmentation Policies from Data"] = "https://arxiv.org/abs/1805.09501"
module_link["image_augmentation-nas_svhn"] = "https://aihub.cloud.google.com/p/products%2F5eef4456-8aa9-47d7-84db-00a7faa6662a"
module_link["image_augmentation-nas_imagenet"] = "https://aihub.cloud.google.com/p/products%2Fdaf153b0-cc1b-4f27-a037-5bd3d9fff224"
module_link["image_augmentation-nas_cifar"] = "https://aihub.cloud.google.com/p/products%2F822fa7f1-2207-4645-bcab-b7b916dae368"



# paper_module["AutoAugment: Learning Augmentation Policies from Data"] = [
#     "image_augmentation-nas_svhn", 
#     "image_augmentation-nas_imagenet", 
#     "image_augmentation-nas_cifar"
# ]
# module_paper["image_augmentation-nas_svhn"] = ["AutoAugment: Learning Augmentation Policies from Data"]

In [3]:
len(paper_module)

29

Noted that: imagenet-amoebanet_a_n18_f448-classification and imagenet-amoebanet_a_n18_f448-feature_vector have 2 references: regularized evolution for image classifier architecture search & learning transferable architectures for scalable image recognition.

According to our implementation, the paper "regularized ..." will shown as "arXiv:1802.01548". That't not very important. We can always make the change manually. 

In [7]:
len(module_paper)

222

In [10]:
from pprint import pprint
pprint(module_paper)

{'Wiki-words-250': ['Efficient Estimation of Word Representations in Vector '
                    'Space'],
 'Wiki-words-250-with-normalization': ['Efficient Estimation of Word '
                                       'Representations in Vector Space'],
 'Wiki-words-500': ['Efficient Estimation of Word Representations in Vector '
                    'Space'],
 'Wiki-words-500-with-normalization': ['Efficient Estimation of Word '
                                       'Representations in Vector Space'],
 'bert_cased_L-12_H-768_A-12': ['BERT:Pre-training of Deep Bidirectional '
                                'Transformers for LanguageUnderstanding'],
 'bert_cased_L-24_H-1024_A-16': ['BERT:Pre-training of Deep Bidirectional '
                                 'Transformers for LanguageUnderstanding'],
 'bert_chinese_L-12_H-768_A-12': ['BERT:Pre-training of Deep Bidirectional '
                                  'Transformers for LanguageUnderstanding'],
 'bert_multi_cased_L-12_H-768_A-12':

## 8 modules are not in there: 

#### 3 models don't have any reference info: 

(They don't clarify their references. Though their webpages look like the previous three, but I don't regard so for certain. )

image_augmentation-crop_rotate_color 1 https://aihub.cloud.google.com/p/products%2Fcd9eda73-8cc4-4dbc-aa62-c97e98d657af

image_augmentation-crop_color 1 https://aihub.cloud.google.com/p/products%2F83bb37d7-5060-4ba7-85e4-6c00a582e789

image_augmentation-flipx_crop_rotate_color 1 https://aihub.cloud.google.com/p/products%2F2012b39c-9bd7-4670-bbc0-190c89405778

#### 2 models clarify their referred stuff: 
faster_rcnn-openimages_v4-inception_resnet_v2 1 https://aihub.cloud.google.com/p/products%2F41b42dfa-e600-4a73-a425-7c5c4d511c3c
* Only has some referred models and datasets: *FasterRCNN+InceptionResNetV2 network trained on Open Images V4*

openimages_v4-ssd-mobilenet_v2 1 https://aihub.cloud.google.com/p/products%2F8c6878ba-d32d-411d-bac2-2f884b748c4f
* Only has some referred models and datasets: *SSD+MobileNetV2 network trained on Open Images V4*

#### 3 models don't have any reference info: 

ganeval-cifar10-convnet 1 https://aihub.cloud.google.com/p/products%2F110689d8-c594-49d6-aef4-9fc91c9c2ab4

random-nnlm-en-dim50 1 https://aihub.cloud.google.com/p/products%2Fb0766fd5-82b5-48e3-9d6f-26c5d2601d5f

random-nnlm-en-dim128 1 https://aihub.cloud.google.com/p/products%2Fa52e9466-f691-4f91-a62e-f6e39eb967f8

-------------------------